<span style="color:Orange;font-weight:700;font-size:50px">
<b>Stock Price Prediction<b>
</span>

<span style="color:orange;font-weight:700;font-size:30px"><b>Project Description:<b></span>

<span style=";font-weight:700;font-size:25px">        
        As any of us may imagine, the market is volatile and, more often than not, unpredictable. For decades, researchers have experimented using time-series data to forecast future values. Market movement is influenced by a variety of factors, only a few of which can be measured, such as historical stock data, trading volume, and current pricing. Fundamental elements such as a firm's intrinsic worth, assets,quarterly performance, recent investments, and strategy, among others, influence traders' faith in the company and hence the stock price. Only a handful of these can be properly included into a mathematical model. As a result, stock price prediction using is difficult and inaccurate in several ways. It is practically difficult to predict whether a pandemic or war will devastate or increase the stock market in the next weeks. Instead of focusing on precisely matching real values, analysts instead focus on creating short-term projections to generate a probabilistic assessment of what the market "might" look like in the near future.
        Here, In our project we have used Artificial Neural networks to solve this Time Series problem. LSTM(Long short-time memory) layers are used in the hidden layers for our analysis.
        </span>

<span style="color:orange;font-weight:700;font-size:30px"><b>Data description:<b></span>

<span style=";font-weight:700;font-size:25px">
        Data set contains 8500 files where each file has data of a company over few years and not all of them are used. Companies are chosen on what comparisons are to be done to explain the sectorial performance. Each file contains 7 columns which are Date, Open, High, Low, Close, Volume, Openlnt. There are 6 Integer columns and 1 dat column.
</span>        
        

            

<span style="color:Orange;font-weight:700;font-size:30px">
<b>Creation of  Stock Industry list csv<b>
</span>

<span style="color:;font-weight:700;font-size:25px">

Here,the Stock industry data is represented and we have dropped few of the unnecessary columns and data.
The given data is  divided on the basis of  the companies into 96 different sectors. This was done so we can perform sectorial analysis.
</span>





In [1]:



import os
import pandas as pd

Path = "Stocks/"
filelist = os.listdir(Path)
df_info = pd.read_csv("stocks-list.csv")
df = pd.DataFrame(columns=['Symbol','Name','Industry'])
for x in filelist:
    symbol = x.split('.')[0].upper()
    try:
        pd.read_csv(Path + x)
        if symbol in df_info['Symbol'].values:
            name = list(df_info.loc[df_info['Symbol'] == symbol].iloc[0])[1]
            name = name.replace('"','')
            industry = list(df_info.loc[df_info['Symbol'] == symbol].iloc[0])[2]
            industry = industry.replace('"','')
            if '\"'in name or '\"' in industry: 
                print((name),(industry))
            df2 = pd.DataFrame({'Symbol': [symbol], 'Name': [name], 'Industry':[industry]})
            df = pd.concat([df, df2], ignore_index = True, axis = 0)
    except:
        pass

df.to_csv('stock_industry_list.csv', index=False)

<span style=color:;font-weight:700;font-size:25px> Creating a csv file with a list of sectors with number of companies in each of them </span>

In [ ]:
# creation of number_of_companies_in_each_industry.csv

import pandas as pd
from collections import defaultdict

df = pd.read_csv("stock_industry_list.csv")
# print(len(df['Industry'].unique()))
df1 = (df.pivot_table(columns=['Industry'], aggfunc='size'))
df1.to_csv('number_of_companies_in_each_industry.csv')



<span style="color:Orange;font-weight:700;font-size:30px">
<b>Functions for creating the database and tables<b>
</span>

<span style=";font-weight:700;font-size:25px">

We have created functions to create the tables and insert the data into the tables.
</span>

In [2]:
# creating the database

from email import header
from locale import normalize
from operator import imod
from unicodedata import category
from numpy import product
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows




<span style="color:Orange;font-weight:700;font-size:30px">
Creating Sector table
</span>
<span style=";font-weight:700;font-size:25px">

The Sector table is created using the  columns sector_id and sector_name.Sector_id is considered as Primary Key here.
</span>

In [ ]:
############### Creating Sector table ##################

conn = create_connection("stocks.db")
sql = """
    create table if not exists Sector(
        SectorName Text not null ,
        SectorID integer not null primary key
    );
"""
create_table(conn,sql)

df = pd.read_csv("stock_industry_list.csv")
df = df[["Industry"]]
df = df.drop_duplicates()
df = df.reset_index(drop=True)


# print(df.shape)


cur = conn.cursor()


for index, row in df.iterrows():
    cur.execute("INSERT INTO Sector (SectorName,SectorID) values(?,?)", (row.Industry,index+1))
    
    

conn.commit()

<span style="color:orange;font-weight:700;font-size:30px">Creating Company table</span>

<span style=";font-weight:700;font-size:25px">

The Company table is created using the  columns company_id ,company_name and sector_id as foreign key.
Using the create_table function that we create the table.


Using the iterrows() function we inserted the values from the data frame into the sql table.
</span>

In [ ]:
############### Creating Company table ##################

conn = create_connection("stocks.db")
sql = """
    create table if not exists Company(
        CompanyId integer not null ,
        CompanySymbol Text not null,
        CompanyName Text not null,
        SectorID integer not null
    );
"""
create_table(conn,sql)


df = pd.read_csv("stock_industry_list.csv")
# print(df)

data = dict(cur.execute("select * from Sector"))
# print(data)

df['SectorID'] = [data[i] for i in df["Industry"]]


# print(df)
# exit()

for index, row in df.iterrows():
    execute_sql_statement(f'INSERT INTO Company(CompanyID,CompanySymbol,CompanyName,SectorID) values({index},"{row.Symbol}","{row.Name}",{row.SectorID})', conn)
    # cur.execute("INSERT INTO Company(CompanyID,CompanySymbol,SectorID) values(?,?,?)", (index,row.Symbol,row.SectorID))
    # print(index+1,row.Symbol,row.SectorID)
    
    

conn.commit()

<span style="color:orange;font-weight:700;font-size:30px">Creating table for each sector and inserting respective companies into database</span>

<span style=";font-weight:700;font-size:25px">Each table belongs to single sector. Each table have all the data of every company that belongs to that sector . In this tables it has companyid, date, open price, closed price, high, low, volume .</span>

In [ ]:
############### Creating Sector table ##################

# execute_sql_statement('Drop table Insurance', conn)
# execute_sql_statement('Drop table Biotechnology', conn)
# execute_sql_statement('Drop table Banks', conn)
# execute_sql_statement('Drop table Beverages', conn)
# execute_sql_statement('Drop table Pharmaceuticals', conn)
# execute_sql_statement('Drop table Software', conn)
# execute_sql_statement('Drop table Entertainment', conn)
# execute_sql_statement('Drop table Chemicals', conn)
# execute_sql_statement('Drop table "Life Sciences Tools & Services"', conn)
# execute_sql_statement('Drop table Life_Sciences_Tools_and_Services', conn)
# exit()

# read stock industry list
df = pd.read_csv("stock_industry_list.csv")
sector_dict = defaultdict(lambda : [])
for index, row in df.iterrows():
    sector_dict[row['Industry']].append(row['Symbol'])
    
for sector in sector_dict.keys():
    sector_id = execute_sql_statement(f'Select SectorID from Sector where SectorName = "{sector}"', conn)[0][0]
    # sector = sector.replace(' ', '_')
    # sector = sector.replace('&', 'and')
    sql = f"""
        create table if not exists "{sector}"(
            CompanyId integer not null ,
            SectorID integer not null,
            Date date not null,
            Open real not null,
            High real not null,
            Low real not null,
            Close real not null,
            Volume integer not null
        );
    """
    create_table(conn,sql)
    conn.commit()
    for company_symbol in sector_dict[sector]:
        company_id = execute_sql_statement(f'Select CompanyId from Company where CompanySymbol = "{company_symbol}"', conn)[0][0]
        # try:
        fp = open(f'Stocks/{company_symbol.lower()}.us.txt', 'r')
        # except FileNotFoundError:
            # continue
        # headers
        fp.readline().strip()
        
        line = fp.readline().strip()
        while line != '':
            row_items = line.split(',')
            execute_sql_statement(f'INSERT INTO "{sector}" values({company_id},{sector_id},"{row_items[0]}",{row_items[1]},{row_items[2]},{row_items[3]},{row_items[4]},{row_items[5]})', conn)
            
            line = fp.readline().strip()
        
        fp.close()
    conn.commit()
    # exit()

<span style="color:orange;font-weight:700;font-size:30px">Visualising the data</span>

<span style=";font-weight:700;font-size:25px">
sectoriAL analysis
</span>

In [5]:
#visualisation
import os
from numpy import datetime64

import pandas as pd
import sqlite3
from sqlite3 import Error
import matplotlib.pyplot as plt
import seaborn as sns

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn



def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows
conn = create_connection('stocks.db')


df1 = pd.read_sql_query('SELECT Date,((Open+High+Low+Close)/4) AS AverageStockprice FROM "Semiconductors & Semiconductor Equipment"  WHERE CompanyID = 1629', conn)#intel
df1 = df1.astype({'Date':datetime64,'AverageStockprice':float})
df2 = pd.read_sql_query('SELECT Date,((Open+High+Low+Close)/4) AS AverageStockprice FROM "Semiconductors & Semiconductor Equipment"  WHERE CompanyID = 2243', conn)#nvidia
df2 = df2.astype({'Date':datetime64,'AverageStockprice':float})
df3 = pd.read_sql_query('SELECT Date,((Open+High+Low+Close)/4) AS AverageStockprice FROM "Semiconductors & Semiconductor Equipment"  WHERE CompanyID = 163', conn)#amd
df3 = df3.astype({'Date':datetime64,'AverageStockprice':float})
plt.figure(figsize = (15,8))
sns.lineplot(x = 'Date', y = 'AverageStockprice', data = df3, color = 'r')
sns.lineplot(x = 'Date', y = 'AverageStockprice', data = df2, color = 'g')
sns.lineplot(x = 'Date', y = 'AverageStockprice', data = df1, color = 'b')
plt.legend(labels=["AMD","NVIDIA","INTEL"])
plt.show()



In [ ]:
df1 = pd.read_sql_query('SELECT Date,((Open+High+Low+Close)/4) AS AverageStockprice FROM "Automobiles"  WHERE CompanyID = 1146', conn)#ford
df1 = df1.astype({'Date':datetime64,'AverageStockprice':float})
df2 = pd.read_sql_query('SELECT Date,((Open+High+Low+Close)/4) AS AverageStockprice FROM "Automobiles"  WHERE CompanyID = 1357', conn)#gm
df2 = df2.astype({'Date':datetime64,'AverageStockprice':float})
df3 = pd.read_sql_query('SELECT Date,((Open+High+Low+Close)/4) AS AverageStockprice FROM "Automobiles"  WHERE CompanyID = 3127', conn)#tesla
df3 = df3.astype({'Date':datetime64,'AverageStockprice':float})

plt.figure(figsize = (15,8))
sns.lineplot(x = 'Date', y = 'AverageStockprice', data = df3, color = 'r', legend = 'full')
sns.lineplot(x = 'Date', y = 'AverageStockprice', data = df2, color = 'g', legend = 'full')
sns.lineplot(x = 'Date', y = 'AverageStockprice', data = df1, color = 'b', legend = 'full')
plt.legend(labels=["Tesla","General Motors","Ford"])
plt.show()

In [ ]:
f1 = pd.read_sql_query('SELECT Date,((Open+High+Low+Close)/4) AS AverageStockprice FROM "Oil, Gas & Consumable Fuels"  WHERE CompanyID = 866', conn)#ford
df1 = df1.astype({'Date':datetime64,'AverageStockprice':float})
df2 = pd.read_sql_query('SELECT Date,((Open+High+Low+Close)/4) AS AverageStockprice FROM "Oil, Gas & Consumable Fuels"  WHERE CompanyID = 3434', conn)#gm
df2 = df2.astype({'Date':datetime64,'AverageStockprice':float})
df3 = pd.read_sql_query('SELECT Date,((Open+High+Low+Close)/4) AS AverageStockprice FROM "Oil, Gas & Consumable Fuels"  WHERE CompanyID = 757', conn)#tesla
df3 = df3.astype({'Date':datetime64,'AverageStockprice':float})

plt.figure(figsize = (15,8))
sns.lineplot(x = 'Date', y = 'AverageStockprice', data = df3, color = 'r', legend = 'full')
sns.lineplot(x = 'Date', y = 'AverageStockprice', data = df2, color = 'g', legend = 'full')
sns.lineplot(x = 'Date', y = 'AverageStockprice', data = df1, color = 'b', legend = 'full')
plt.legend(labels=["Chevron Corporation","Exxon Mobil Corp","Conoco Phillips"])
plt.show()

<span style="color:orange;font-weight:700;font-size:30px">Building a model and predicting the price of the stock</span>


<span style=";font-weight:700;font-size:25px">The given data from the tables is divided into train and test data sets. To get the maximum accuracy we have used 90% of the data as training data set and 10% of it as test data set.
The Artifical Neural Networks is used to predict the prices on every day , starting from the 90th day as The first 90 days are used as intial inputs and we have seen the trends to change after evervy 90 days on average.In the Arificial Neural Networks we have used the LSTM(Long short-time memory) layer as our  hidden layer. Four hidden layers are used here and a dense layer for the output. In the end we  used r2 score to know the accuracy of the model and plotted the predicted values vs actual values over.</span>    
 

In [9]:
# prediction of stock prices

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import LSTM
from keras. layers import Dense



#pulling the data from database and inserting in a dataframe
conn = sqlite3.connect('stocks.db')

sql_query = pd.read_sql_query ('''SELECT * FROM Banks where CompanyId = 358 order by Date''', conn)

df = pd.DataFrame(sql_query, columns = ['Date', 'Open', 'High' ,'Low', 'Close', 'Volume'])

train = df.iloc[:,1:2].values



scaler = MinMaxScaler(feature_range = (0,1))
temp_dataset = scaler.fit_transform(train)


X_train , y_train, X_test, y_test = [],[],[],[]
for i in range(90,df.shape[0]):
    if i < (df.shape[0]-90)*0.9:
        X_train.append(temp_dataset[i-90:i-1,0])
        y_train.append(temp_dataset[i,0])
    else:
        X_test.append(temp_dataset[i-90:i-1,0])
        y_test.append(temp_dataset[i,0])
    
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)




# model Building
model = Sequential()
model.add(LSTM(units = 30, return_sequences= True, input_shape = (X_train.shape[1], 1)))
model.add(LSTM(units = 70, return_sequences= True))
model.add(LSTM(units = 60, return_sequences= True))
model.add(LSTM(units = 40))

model.add(Dense(units = 1))


#Fitting the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=70)


predicted_stock_price = model.predict(X_test)

print(r2_score(y_test, predicted_stock_price))



plt.plot(y_test, color = 'red', label = "Actual BOFA Stock Price")
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted BOFA Stock Price')
plt.title('BOFA Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('BOFA Stock Price')
plt.legend()
plt.show()





ModuleNotFoundError: No module named 'tensorflow'